<a href="https://colab.research.google.com/github/Nishita-shah1/ML/blob/main/Nis_spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np
print(type(pd.DataFrame))

df = pd.read_csv('spam_detection.csv',encoding='latin1')
print(df.head())

In [ ]:
df.shape

data cleaning


In [ ]:
df.info()

In [ ]:
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [ ]:
df.info()

In [ ]:
df.rename(columns={'v1':'target','v2':'text'},inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder # this converts categorial data to numerical
encoder=LabelEncoder()

In [ ]:
df['target']=encoder.fit_transform(df['target'])

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(keep='first', inplace=True)


In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

EDA

In [ ]:
df['target'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.pie(df['target'].value_counts(),labels=['ham','spam'],autopct='%0.2f')
plt.show()

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')


In [ ]:
df['num_char']=df['text'].apply(len)

In [ ]:
df.head()

In [ ]:
# num of words
df['num_words']=df['text'].apply(lambda x:len(nltk.word_tokenize(x)))
df.head()

In [ ]:
#number of sentences
df['num_sentence']=df['text'].apply(lambda x:len(nltk.sent_tokenize(x)))
df.head()

In [ ]:
df[['num_char','num_words','num_sentence']].describe()


In [ ]:
#SPAM
df[df['target']==0][['num_char','num_words','num_sentence']].describe()

In [ ]:
#ham
df[df['target']==1][['num_char','num_words','num_sentence']].describe()

In [ ]:
import seaborn as sns


In [ ]:
sns.histplot(df[df['target']==0]['num_char'])
sns.histplot(df[df['target']==1]['num_char'],color='red')

In [ ]:
sns.histplot(df[df['target']==0]['num_words'])
sns.histplot(df[df['target']==1]['num_words'],color='red')

In [ ]:
sns.pairplot(df,hue='target')

In [ ]:

# Filter only the numerical columns
numeric_df = df.select_dtypes(include=[float, int])

# Compute the correlation matrix
corr_matrix = numeric_df.corr()
# Generate the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True)
plt.show()

Data preprocessing
1. lower case
2. tokenization
3. removing special characters
4. removing stop words and punctuation
5. stemming

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords.words('english')

In [ ]:
import string
string.punctuation

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
ps.stem('loving')

In [ ]:
def transform_text(text):
  text=text.lower()
  text=nltk.word_tokenize(text)
  y=[]
  for i in text:
    if i.isalnum():
      if i not in stopwords.words('english') and i not in string.punctuation:
        y.append(ps.stem(i))
  return " ".join (y)

In [ ]:
transform_text('did you like my ML presentation')

In [ ]:
df['transformed_text']=df['text'].apply(transform_text)

In [ ]:
df.head()

In [ ]:
from wordcloud import WordCloud

wc = WordCloud(width=500, height=500, min_font_size=10, background_color='white')


In [ ]:
spam_wc=wc.generate(df[df['target']==1]['transformed_text'].str.cat(sep=""))

In [ ]:
plt.imshow(spam_wc)

In [ ]:
ham_wc=wc.generate(df[df['target']==0]['transformed_text'].str.cat(sep=""))

In [ ]:
plt.imshow(ham_wc)

In [ ]:
import pandas as pd
from collections import Counter
spam_corpus = []
for msg in df[df['target'] == 1]['transformed_text'].tolist():
    for word in msg.split():
        spam_corpus.append(word)

#Check the content of spam_corpus
print("First 10 words in spam_corpus:", spam_corpus[:10])

# Create a Counter object
counter = Counter(spam_corpus)

# Get the 30 most common elements
most_common = counter.most_common(30)

# Debug: Check the most common words
print("Most common words:", most_common)

# Create a DataFrame
df_common = pd.DataFrame(most_common, columns=['Word', 'Frequency'])

print(df_common)


In [ ]:
print(type(pd.DataFrame))


In [ ]:
len(spam_corpus)
print(spam_corpus)

In [ ]:
# Plotting
plt.figure(figsize=(12, 8))
sns.barplot(x='Word', y='Frequency', data=df_common)
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.xlabel('Word')
plt.ylabel('Frequency')
plt.title('Top 30 Most Common Words in Spam Corpus')
plt.show()


**Model** **Building**
we will use naive bayes model here as it is better for textual based model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv=CountVectorizer()
tfidf=TfidfVectorizer(max_features=3000)

In [ ]:
 x=tfidf.fit_transform(df['transformed_text']).toarray()
 print (x)

In [ ]:
x.shape

In [ ]:
y=df['target'].values

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()


In [ ]:
gnb.fit(x_train,y_train)
y_pred1=gnb.predict(x_test)
print(accuracy_score(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))
print(precision_score(y_test,y_pred1))

In [ ]:
mnb.fit(x_train,y_train)
y_pred2=mnb.predict(x_test)
print(accuracy_score(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))
print(precision_score(y_test,y_pred2))

In [ ]:
bnb.fit(x_train,y_train)
y_pred3=bnb.predict(x_test)
print(accuracy_score(y_test,y_pred3))
print(confusion_matrix(y_test,y_pred3))
print(precision_score(y_test,y_pred3))

In [ ]:
# tfidf-->mnd


In [ ]:
''' model improve
1. stacking
2. voting'''


In [ ]:
import pickle
pickle.dump(tfidf,open('vectorizer.pkl','wb'))
pickle.dump(mnb,open('model.pkl','wb'))